# Setup

In [1]:
import pandas as pd
import os 

In [2]:
DATE = '2019_01'

In [3]:
points_df = pd.read_csv(f'../data/points_data.csv')
op_data = pd.read_csv(f'../data/opData/op_data_{DATE}.csv')

In [4]:
print(points_df.shape)
points_df.columns

(772, 32)


Index(['Unnamed: 0', 'point_key', 'point_label', 'is_single_operator',
       'point_tooltip', 'point_eic_code', 'control_point_type', 'tp_map_x',
       'tp_map_y', 'point_type', 'commercial_type', 'import_from_country_key',
       'import_from_country_label', 'has_virtual_point', 'virtual_point_key',
       'virtual_point_label', 'has_data', 'is_planned', 'is_interconnection',
       'is_import', 'infrastructure_key', 'infrastructure_label',
       'is_cross_border', 'eu_crossing', 'is_invalid', 'is_macro_point',
       'is_cam_relevant', 'is_pipe_in_pipe', 'is_cmp_relevant', 'id',
       'data_set', 'url'],
      dtype='object')

In [5]:
print(op_data.shape)
op_data.columns

(10000, 14)


Index(['Unnamed: 0', 'direction_key', 'flow_status', 'id', 'id_point_type',
       'operator_key', 'period_from', 'period_to', 'period_type', 'point_key',
       'point_label', 'tso_eic_code', 'unit', 'value'],
      dtype='object')

point_type :    
*Distribution Point ExtEU                                          128 --> Ignore*    
*Storage point                                                     122 --> Ignore*   
Cross-Border **Transmission** IP within EU                            106    
*Aggregated Point - Final Consumers                                 60 -->  Ignore*    
*Distribution Point                                                 48 --> Ignore*    
**LNG Entry point                                                    46**    
Cross-Border **Transmission** IP between EU and Non-EU (import)        38    
*Aggregated production point - TP                                   37 --> Ignore*     
**Transmission** Point                                                 32    
*Trading Point                                                      29 --> Ignore*   
Cross-Border **Transmission** IP between EU and Non-EU                 27    
Cross-Border **Transmission** IP Non-EU                                25    
*Storage point ExtEU                                                21*    
*Cross-Border Storage IP within EU                                  17*     
*Storage IP Non-EU                                                  13*    
Cross-Border **Transmission** IP between EU and ExtEU                   7    
*Aggregated production point - TP ExtEU                              5 --> Ignore*    
**Transmission** Point ExtEU                                            3    
**LNG Entry point ExtEU                                               2**    
Cross-Border **Transmission** IP between ExtEU and Non-EU (import)      2    
*Aggregated Point - Final Consumers ExtEU                            2 --> Ignore*    
*Aggregated production point - SysDev                                1 --> Ignore*   
*Trading Point Ext-EU                                                1 --> Ignore*

Choose to focus on : 
* Transmission points
* LNG entries

In [6]:
def filter_point_type(keyword,df,column):
    return df[df[column].str.contains(keyword)]


In [7]:
trans_points_df = filter_point_type("Transmission",points_df,'point_type')
lng_points_df = filter_point_type("LNG",points_df,'point_type')

print(f"Transmission points : {len(trans_points_df)}")
print(f"LNG points : {len(lng_points_df)}")

Transmission points : 240
LNG points : 48


## Geographical data of LNG points

In [8]:
# results_df = pd.read_csv('../data/results.csv')
# results_df.columns

Index(['Unnamed: 0', 'geojson_name', 'point_label', 'point_type', 'geojson_id',
       'tp_map_x', 'tp_map_y', 'latitude', 'longitude'],
      dtype='object')

In [9]:
# results_df.shape

(126, 9)

In [10]:
# res_lng_points = filter_point_type("LNG",results_df,'point_type')
# res_lng_points.shape

(23, 9)

NOTE : Only 23 out of 48 points appear in results  

==> Check with OSM

In [11]:
# osm_df = pd.read_csv('../../OSMData/INET_LNGs.csv',sep=';')
# osm_df.columns
# print(f"OSM LNG points : {len(osm_df)}")
# print(f"Columns : {osm_df.columns}")

OSM LNG points : 29
Columns : Index(['id', 'name', 'source_id', 'node_id', 'lat', 'long', 'country_code',
       'comment', 'param', 'uncertainty', 'method', 'tags'],
      dtype='object')


In [12]:
# print("points_df LNG points : ")
# print(lng_points_df["point_label"].to_string(index=False))

# print("results_df LNG points : ")
# print(res_lng_points["point_label"].to_string(index=False))

# print("OSM LNG points : ")
# print(osm_df[["name","lat","long"]].to_string(index=False))

PROCESS : After analysing by hand and via differen sources, we obtained information about all points from the dataset. The process is documented in OneNote and the final dataset is LNG_points_info.csv

In [13]:
lng_points_df.head()
print(lng_points_df.shape)

(48, 32)


In [14]:
lng_points_info = pd.read_excel('../data/points/LNG_points_info.xlsx')
lng_points_info.rename(columns={'points_label':'point_label','LNG Entry Point':'mapped_name','Longitude':'lon','Latitude':'lat'},inplace=True)
lng_points_info.head()

,Country,index,point_label,is_operational,mapped_name,lat,lon
0,Greece,390,Alexandroupolis LNG,False,Alexandroupolis,40833,25872.00
1,Italy,458,OLT LNG / Livorno,True,Toscana,43067,10067.00
2,Poland,460,Swinoujscie,True,Swinoujscie,53917,14250.00
3,United Kingdom,462,Isle of Grain,True,Isle of Grain,51450,-0.75
4,Estonia,463,Paldiski LNG,False,Paldiski LNG,59367,24050.00


In [15]:
# PRocess points because excell removed the floating point
lng_points_info['lat'] = lng_points_info['lat'].apply(lambda x: x/1000 if x>=0 else x)
lng_points_info['lon'] = lng_points_info['lon'].apply(lambda x: x/1000 if x>=0 else x)

In [16]:
lng_points_df = lng_points_df.merge(lng_points_info[['point_label','mapped_name','Country', 'lat', 'lon','is_operational']], on='point_label', how='left')

In [17]:
print(lng_points_df.shape)
print(lng_points_info.shape)

(48, 37)
(48, 7)


In [18]:
lng_points_df.columns

Index(['Unnamed: 0', 'point_key', 'point_label', 'is_single_operator',
       'point_tooltip', 'point_eic_code', 'control_point_type', 'tp_map_x',
       'tp_map_y', 'point_type', 'commercial_type', 'import_from_country_key',
       'import_from_country_label', 'has_virtual_point', 'virtual_point_key',
       'virtual_point_label', 'has_data', 'is_planned', 'is_interconnection',
       'is_import', 'infrastructure_key', 'infrastructure_label',
       'is_cross_border', 'eu_crossing', 'is_invalid', 'is_macro_point',
       'is_cam_relevant', 'is_pipe_in_pipe', 'is_cmp_relevant', 'id',
       'data_set', 'url', 'mapped_name', 'Country', 'lat', 'lon',
       'is_operational'],
      dtype='object')

In [19]:
missing_countries = lng_points_df[lng_points_df['lat'].isna() | lng_points_df['lon'].isna()]['point_label']
# print(missing_countries)
# 49    Baltic Energy Gate => CHOOSE TO IGNORE
lng_points_df = lng_points_df[lng_points_df['Country'].notna()]

In [52]:
lng_points_df.to_csv('../data/points/lng_points.csv',index=False)

Other remarks :
* 2 points have keys that are resp. ITP and VTP, those points correspond to Alexandropoulis LNG and TVB

# LNG entries operational data

In [20]:
lng_points_df.head()

,Unnamed: 0,point_key,point_label,is_single_operator,point_tooltip,point_eic_code,control_point_type,tp_map_x,tp_map_y,point_type,...,is_pipe_in_pipe,is_cmp_relevant,id,data_set,url,mapped_name,Country,lat,lon,is_operational
0,390,ITP-00424,Alexandroupolis LNG,1,NaN,?,P_P_INCOUN_IN_LNG,51.55,-67.24,LNG Entry point,...,False,False,2ITP-00424,2,https://transparency.entsog.eu/api/v1/connecti...,Alexandroupolis,Greece,40.833,25.872,False
1,458,LNG-00004,OLT LNG / Livorno,0,NaN,21Z000000000275G,O_P_INCOUN_IN_LNG,-29.75,-58.33,LNG Entry point,...,False,False,2LNG-00004,2,https://transparency.entsog.eu/api/v1/connecti...,Toscana,Italy,43.067,10.067,True
2,460,LNG-00006,Swinoujscie,0,NaN,21Z000000000454G,O_P_INCOUN_IN_LNG,-7.91,10.01,LNG Entry point,...,False,False,2LNG-00006,2,https://transparency.entsog.eu/api/v1/connecti...,Swinoujscie,Poland,53.917,14.250,True
3,462,LNG-00008,Isle of Grain,0,NaN,21YIG-EC-------X,O_P_INCOUN_IN_LNG_ExtEU,-79.85,-11.39,LNG Entry point ExtEU,...,False,False,2LNG-00008,2,https://transparency.entsog.eu/api/v1/connecti...,Isle of Grain,United Kingdom,51.450,-0.750,True
4,463,LNG-00010,Paldiski LNG,0,NaN,?,P_P_INCOUN_IN_LNG,41.92,52.64,LNG Entry point,...,False,False,2LNG-00010,2,https://transparency.entsog.eu/api/v1/connecti...,Paldiski LNG,Estonia,59.367,24.050,False


In [21]:
lng_points_df.columns

Index(['Unnamed: 0', 'point_key', 'point_label', 'is_single_operator',
       'point_tooltip', 'point_eic_code', 'control_point_type', 'tp_map_x',
       'tp_map_y', 'point_type', 'commercial_type', 'import_from_country_key',
       'import_from_country_label', 'has_virtual_point', 'virtual_point_key',
       'virtual_point_label', 'has_data', 'is_planned', 'is_interconnection',
       'is_import', 'infrastructure_key', 'infrastructure_label',
       'is_cross_border', 'eu_crossing', 'is_invalid', 'is_macro_point',
       'is_cam_relevant', 'is_pipe_in_pipe', 'is_cmp_relevant', 'id',
       'data_set', 'url', 'mapped_name', 'Country', 'lat', 'lon',
       'is_operational'],
      dtype='object')

In [22]:
op_lng_points_df = lng_points_df[lng_points_df['is_operational']]
print(op_lng_points_df.shape)


(30, 37)


In [23]:
print(lng_points_df[lng_points_df['is_operational']][['point_label','point_key']])

                                 point_label  point_key
1                          OLT LNG / Livorno  LNG-00004
2                                Swinoujscie  LNG-00006
3                              Isle of Grain  LNG-00008
5                                  Barcelona  LNG-00012
6                                     Bilbao  LNG-00013
7                                Agia Triada  LNG-00014
8   Cavarzere (Porto Levante / Adriatic LNG)  LNG-00015
9                              Zeebrugge LNG  LNG-00017
10                                    Huelva  LNG-00018
11                                Panigaglia  LNG-00019
12                                  Mugardos  LNG-00021
13                                 Cartagena  LNG-00022
14                                   Sagunto  LNG-00023
15                       Montoir de Bretagne  LNG-00024
16                                     Sines  LNG-00026
17                         Gate Terminal (I)  LNG-00027
18                       Fos (Tonkin/Cavaou)  LN

In [24]:
lng_keys = lng_points_df['point_key'].unique()
print(lng_keys)

['ITP-00424' 'LNG-00004' 'LNG-00006' 'LNG-00008' 'LNG-00010' 'LNG-00012'
 'LNG-00013' 'LNG-00014' 'LNG-00015' 'LNG-00017' 'LNG-00018' 'LNG-00019'
 'LNG-00021' 'LNG-00022' 'LNG-00023' 'LNG-00024' 'LNG-00026' 'LNG-00027'
 'LNG-00029' 'LNG-00030' 'LNG-00032' 'LNG-00034' 'LNG-00038' 'LNG-00039'
 'LNG-00041' 'LNG-00042' 'LNG-00044' 'LNG-00045' 'LNG-00047' 'LNG-00049'
 'LNG-00056' 'LNG-00057' 'LNG-00058' 'LNG-00059' 'LNG-00060' 'LNG-00064'
 'VTP-00043' 'LNG-00068' 'LNG-00011' 'LNG-00072' 'LNG-00066' 'LNG-00075'
 'LNG-00069' 'LNG-00071' 'LNG-00016' 'LNG-00078' 'LNG-00080']


In [25]:
lng_op_data = op_data[op_data['point_key'].isin(lng_keys)]

In [26]:
op_lng_keys = lng_op_data['point_key'].unique()

NOTE : All keys from op data are matched in lng_keys

**TODO** Create a dictionnary with keys as point_keys to retrieve data. Each entry should contain : 
* point_label
* coordinates
* is operational
* Country

Then we must process operational data so that we add the total entry / exit flow for each point. Set all entries/ exits to zero by default

IDEA : Instead of doing one file for each month, do a global file for the year with total entries per month and a total for the year

## Dictionnary for LNG point key

### Point data

In [27]:
# LNG DICTIONNARY
def process_point_data(points_df,dict):
    """ 
    Pre : The points_df should contain the following coluns : point_label,coordinates,is_operational,Country
    """
    # Populate the dictionary
    for _, row in points_df.iterrows():
        point_key = row['point_key']
        dict[point_key] = {
            'point_label': row['point_label'],
            'coordinates': (row['lat'], row['lon']),
            'is_operational': row['is_operational'],
            'Country': row['Country'],
            'total_entries': [0] * 12,  # 12 months, initialized to 0
            'total_exits': [0] * 12     # 12 months, initialized to 0
        }

### Operational data

NOTE : These functions can be re-used for all point types

In [28]:
def process_monthly_data(dict,op_data, date_idx):
    """ 
    Pre : Month index is a two-digit string starting from 01 to 12.
    Post : Add operational data entries to the corresponding point_key entry in 
    the dictionnary
    """
    # Aggregate the entries and exits
    for _, row in op_data.iterrows():
        point_key = row['point_key']
        value = row['value']
        
        # Check flow direction and update accordingly
        if row['direction_key'] == 'entry':
            dict[point_key]['total_entries'][date_idx] += value
        elif row['direction_key'] == 'exit':
            dict[point_key]['total_exits'][date_idx] += value

In [29]:
def process_total_data(dict,dates,point_keys,directory="../data/opData"):
    """
    Pre : Dates is a list of dates following the same format used when encoding
    the csv files (ex : 2019_01, 2019,02,...)
    point_keys is the list of keys we want to process the data of
    Post : A dictionnary with point_key as keys containing point info and tha total 
    entries and exits for each month in the dates input
    """
    for date in dates:
        op_data = pd.read_csv(f'{directory}/op_data_{date}.csv')
        # print(op_data.columns)
        filtered_op_data = op_data[op_data['point_key'].isin(point_keys)]

        process_monthly_data(dict,filtered_op_data,int(date[-2:])-1)

        

In [30]:
# EXAMPLE
# Example using 3 first months of 2019

lng_op_data_2019 = {}
process_point_data(lng_points_df,lng_op_data_2019)
process_total_data(lng_op_data_2019,['2019_01','2019_02','2019_03'],lng_keys) 

print(lng_op_data_2019['LNG-00017'])

{'point_label': 'Zeebrugge LNG', 'coordinates': (51.333, 3.2), 'is_operational': True, 'Country': 'Belgium', 'total_entries': [2249386658.0, 1599130565.0, 3941278926.0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'total_exits': [0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [31]:
# GOOD JOB IT WORKS
#               ====           ====
#             ==    ==       ==    ==
#                         mm
#             ==                     ==
#               ==== ========== ====
#                   ============

## LNG Map

In [32]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from collections import defaultdict

In [33]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [34]:
def update_max(entries,exits,global_max_entry,global_max_exit):
    global_max_entry = max(entries,global_max_entry)
    global_max_exit = max(exits,global_max_exit)

    return global_max_entry,global_max_exit


In [35]:
# MAP WITH SIDE BUTTONS
# def create_lng_map(lng_data):

#     global_max_exit = 0
#     global_max_entry = 0

#     # Initialize the figure
#     fig = go.Figure()
    
#     # Colors for import and export
#     colors = {
#         'import': 'ylorrd',
#         'export': 'blues'
#     }

    
#     # Create traces for each month
#     for month_idx in range(12):
#         entries_lats = []
#         entries_lons = []
#         entries_sizes = []
#         entries_labels = []
        
#         exits_lats = []
#         exits_lons = []
#         exits_sizes = []
#         exits_labels = []

#         no_flow_lats = []
#         no_flow_lons = []
#         no_flow_labels = []
        
#         for point_key, data in lng_data.items():
            
#             lat, lon = data['coordinates']
#             entries_size = data['total_entries'][month_idx]
#             exits_size = data['total_exits'][month_idx]
#             label = data['point_label']

#             # Update total max values to get the color scale right
#             # TODO : Either keep the same scale or change for each month
#             # CURRENT : global max value for the year
#             global_max_entry,global_max_exit = update_max(entries_size,exits_size,global_max_entry,global_max_exit)
            
#             if entries_size > 0:
#                 entries_lats.append(lat)
#                 entries_lons.append(lon)
#                 entries_sizes.append(entries_size)
#                 entries_labels.append(f'{label}: {entries_size:.2e} kWh/d')
                
#             if exits_size > 0:
#                 exits_lats.append(lat)
#                 exits_lons.append(lon)
#                 exits_sizes.append(exits_size)
#                 exits_labels.append(f'{label}: {exits_size:.2e} kWh/d')

#             if entries_size == 0 and exits_size == 0:
#                 no_flow_lats.append(lat)
#                 no_flow_lons.append(lon)
#                 no_flow_labels.append(label)
        
#         # Add entries trace
#         fig.add_trace(go.Scattergeo(
#             locationmode='country names',
#             lon=entries_lons,
#             lat=entries_lats,
#             text=entries_labels,
#             opacity=0.6,
#             marker=dict(
#                 size=[s/10000000 for s in entries_sizes],  # Adjust size scale
#                 color=entries_sizes,
#                 colorscale=colors['import'],
#                 colorbar=dict(
#                     title="Import",
#                     titleside="right",
#                     tickmode="array",
#                     tickvals=[0, global_max_entry],
#                     ticktext=["0", f"{global_max_entry:.2e}"]),
#                 line_width=0.5,
#                 sizemode='area'
#             ),
#             name=f'Entries - {months[month_idx]}',
#             visible=(month_idx == 0)  # Only the first month is visible initially
#         ))
#         # Add exits trace
#         fig.add_trace(go.Scattergeo(
#             locationmode='country names',
#             lon=exits_lons,
#             lat=exits_lats,
#             text=exits_labels,
#             marker=dict(
#                 size=[s/10000000 for s in exits_sizes],  # Adjust size scale
#                 color=exits_sizes,
#                 colorscale=colors['export'],
#                 colorbar=dict(
#                     title="Export",
#                     titleside="right",
#                     tickmode="array",
#                     tickvals=[0, global_max_exit],
#                     ticktext=["0", f"{global_max_exit:.2e}"]),
#                 line_width=0.5,
#                 sizemode='area'
#             ),
#             name=f'Exits - {months[month_idx]}',
#             visible=(month_idx == 0)  # Only the first month is visible initially
#         ))
#         # Add no flow trace
#         fig.add_trace(go.Scattergeo(
#             locationmode='country names',
#             lon=no_flow_lons,
#             lat=no_flow_lats,
#             text=no_flow_labels,
#             marker=dict(
#                 size=5,  # Fixed size for no flow points
#                 color='black',
#                 line_width=0.5,
#                 sizemode='area'
#             ),
#             name=f'No Flow - {months[month_idx]}',
#             visible=(month_idx == 0)  # Only the first month is visible initially
#         ))
    
#     # Create buttons for each month
#     buttons = []
#     for month_idx in range(12):
#         buttons.append(dict(
#             method='restyle',
#             label=f'{months[month_idx]}',
#             args=[{
#                 'visible': [i//3 == month_idx for i in range(36)]
#             }]
#         ))
#     # Update display layout
#     # fig.update_layout(
#     #     margin={"r":0,"t":0,"l":0,"b":0,"autoexpand":True},
#     #     width=800

#     # )
#     fig.update_geos(
#         center = {"lat": 50.5, "lon": 15.2551},
#             projection_scale = 4.3,         
#     )
    
#     # Add buttons to the layout
#     fig.update_layout(
#         updatemenus=[{
#             'buttons': buttons,
#             'direction': 'down',
#             'type': 'buttons',
#         }],
#         title='LNG Imports and Exports',
#         width=1000, height=1000,
#         geo=dict(
#             projection_type='mercator',
#             showland=True,
#             showcountries=True,
#             landcolor='rgb(217, 217, 217)',
#             subunitwidth=1,
#             countrywidth=1,
#             subunitcolor='rgb(255, 255, 255)',
#             countrycolor='rgb(255, 255, 255)'
#         ),
#         coloraxis=dict(
#             colorscale=colors['import'],
#             cmin=0,
#             cmax=global_max_entry,
#             colorbar=dict(title="Flow (kWh/d)")
#         )
#     )
    
#     fig.show()


In [36]:
# MAP WITH SLIDER
def create_lng_map(lng_data):

    global_max_exit = 0
    global_max_entry = 0

    # Initialize the figure
    fig = go.Figure()
    
    # Colors for import and export
    colors = {
        'import': 'ylorrd',
        'export': 'blues'
    }

    
    # Create traces for each month
    for month_idx in range(12):
        entries_lats = []
        entries_lons = []
        entries_sizes = []
        entries_labels = []
        
        exits_lats = []
        exits_lons = []
        exits_sizes = []
        exits_labels = []

        no_flow_lats = []
        no_flow_lons = []
        no_flow_labels = []
        
        for point_key, data in lng_data.items():
            
            lat, lon = data['coordinates']
            entries_size = data['total_entries'][month_idx]
            exits_size = data['total_exits'][month_idx]
            label = data['point_label']

            # Update total max values to get the color scale right
            # TODO : Either keep the same scale or change for each month
            # CURRENT : global max value for the year
            global_max_entry,global_max_exit = update_max(entries_size,exits_size,global_max_entry,global_max_exit)
            
            if entries_size > 0:
                entries_lats.append(lat)
                entries_lons.append(lon)
                entries_sizes.append(entries_size)
                entries_labels.append(f'{label}: {entries_size:.2e} kWh/d')
                
            if exits_size > 0:
                exits_lats.append(lat)
                exits_lons.append(lon)
                exits_sizes.append(exits_size)
                exits_labels.append(f'{label}: {exits_size:.2e} kWh/d')

            if entries_size == 0 and exits_size == 0:
                no_flow_lats.append(lat)
                no_flow_lons.append(lon)
                no_flow_labels.append(label)
        
        # Add entries trace
        fig.add_trace(go.Scattergeo(
            locationmode='country names',
            lon=entries_lons,
            lat=entries_lats,
            text=entries_labels,
            opacity=0.6,
            marker=dict(
                size=[s/10000000 for s in entries_sizes],  # Adjust size scale
                color=entries_sizes,
                colorscale=colors['import'],
                colorbar=dict(
                    title="Import",
                    titleside="right",
                    tickmode="array",
                    tickvals=[0, global_max_entry],
                    ticktext=["0", f"{global_max_entry:.2e}"]),
                line_width=0.5,
                sizemode='area'
            ),
            name=f'Entries - {months[month_idx]}',
            visible=(month_idx == 0)  # Only the first month is visible initially
        ))
        # Add exits trace
        fig.add_trace(go.Scattergeo(
            locationmode='country names',
            lon=exits_lons,
            lat=exits_lats,
            text=exits_labels,
            marker=dict(
                size=[s/10000000 for s in exits_sizes],  # Adjust size scale
                color=exits_sizes,
                colorscale=colors['export'],
                colorbar=dict(
                    title="Export",
                    titleside="right",
                    tickmode="array",
                    tickvals=[0, global_max_exit],
                    ticktext=["0", f"{global_max_exit:.2e}"]),
                line_width=0.5,
                sizemode='area'
            ),
            name=f'Exits - {months[month_idx]}',
            visible=(month_idx == 0)  # Only the first month is visible initially
        ))
        # Add no flow trace
        fig.add_trace(go.Scattergeo(
            locationmode='country names',
            lon=no_flow_lons,
            lat=no_flow_lats,
            text=no_flow_labels,
            marker=dict(
                size=5,  # Fixed size for no flow points
                color='black',
                line_width=0.5,
                sizemode='area'
            ),
            name=f'No Flow - {months[month_idx]}',
            visible=(month_idx == 0)  # Only the first month is visible initially
        ))
    
    # Create slider
    steps = []
    for month_idx in range(12):
        step = dict(
            method='update',
            label=f'{months[month_idx]}',
            args=[{
                'visible': [i//3 == month_idx for i in range(36)]
            }]
        )
        steps.append(step)

    fig.update_geos(
        center = {"lat": 50.5, "lon": 15.2551},
            projection_scale = 4.3,         
    )
    
    # Add slider to the layout
    fig.update_layout(
        sliders = [{
            'active': 0,
            'steps': steps,
            'currentvalue':{"prefix": "Month: "},
            'pad':{"t": 50},
        }],
        
        title='LNG Imports and Exports',
        width=1000, height=1000,
        geo=dict(
            projection_type='mercator',
            showland=True,
            showcountries=True,
            landcolor='rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor='rgb(255, 255, 255)',
            countrycolor='rgb(255, 255, 255)'
        ),
        coloraxis=dict(
            colorscale=colors['import'],
            cmin=0,
            cmax=global_max_entry,
            colorbar=dict(title="Flow (kWh/d)")
        )
    )
    
    fig.show()


Issues & choices:
* Scale
* 

In [37]:
# Assuming lng_points_df and lng_keys are already defined as pandas DataFrame and list

# Initialize the LNG data dictionary
lng_op_data_2019 = {}
process_point_data(lng_points_df, lng_op_data_2019)

# Process the data for the first three months of 2019
process_total_data(lng_op_data_2019, ['2019_01', '2019_02', '2019_03','2019_04','2019_05','2019_06','2019_07','2019_08','2019_09','2019_10','2019_11','2019_12'], lng_keys)


# Create and display the map
create_lng_map(lng_op_data_2019)


In [38]:
# Assuming lng_points_df and lng_keys are already defined as pandas DataFrame and list

# Initialize the LNG data dictionary
lng_op_data_2023 = {}
process_point_data(lng_points_df, lng_op_data_2023)

# Process the data for the year 2023
process_total_data(lng_op_data_2023, ['2023_01', '2023_02', '2023_03', '2023_04', '2023_05', '2023_06', '2023_07', '2023_08', '2023_09', '2023_10', '2023_11', '2023_12'], lng_keys)

# Create and display the map
create_lng_map(lng_op_data_2023)

**TODO :**
* Create a dataset with all entries/exits of LNG terminals per country. This can later be used to compare import/exports per year
* When clicking on a country, we should display the connected countries and what amount of flow went to them
* Compute total country entry/exits for transmission points THEN add the lng entries.
* Removed planned points 

**DISPLAY IDEAS**
* When clicking on the country, display a graph of the proportion of entries/exits to each country over the year
* Bar chart pop up for each country with its entry points
* Use **Dash**

## Yearly evolution map

GPT PROMPT : 

In [40]:
# import pandas as pd
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# def create_interactive_lng_map(lng_keys, lng_points_df, dates, op_data_directory):
#     lng_dict = initialize_lng_dict(lng_points_df)
#     process_total_data(lng_dict, dates, lng_keys, op_data_directory)
    
#     fig = setup_main_map_figure(lng_dict)
#     add_lng_traces(fig, lng_dict)
#     add_slider_to_figure(fig)

#     fig = add_country_click_interaction(fig, lng_dict)
    
#     fig.show()

def initialize_lng_dict(points_df):
    lng_dict = {}
    for _, row in points_df.iterrows():
        point_key = row['point_key']
        lng_dict[point_key] = {
            'point_label': row['point_label'],
            'coordinates': (row['lat'], row['lon']),
            'is_operational': row['is_operational'],
            'Country': row['Country'],
            'total_entries': [0] * 12,  # 12 months, initialized to 0
            'total_exits': [0] * 12     # 12 months, initialized to 0
        }
    return lng_dict

# def process_monthly_data(lng_dict, op_data, date_idx):
#     for _, row in op_data.iterrows():
#         point_key = row['point_key']
#         value = row['value']
#         if row['direction_key'] == 'entry':
#             lng_dict[point_key]['total_entries'][date_idx] += value
#         elif row['direction_key'] == 'exit':
#             lng_dict[point_key]['total_exits'][date_idx] += value

# def process_total_data(lng_dict, dates, point_keys, directory):
#     for date in dates:
#         op_data = pd.read_csv(f'{directory}/op_data_{date}.csv')
#         filtered_op_data = op_data[op_data['point_key'].isin(point_keys)]
#         process_monthly_data(lng_dict, filtered_op_data, int(date[-2:]) - 1)

# def setup_main_map_figure(lng_dict):
#     fig = go.Figure()
#     fig.update_layout(
#         title='LNG Imports and Exports with Interactivity',
#         geo=dict(
#             scope='europe',  # Focus on Europe
#             projection_type='mercator',
#             showland=True,
#             landcolor='rgb(217, 217, 217)',
#             subunitwidth=1,
#             countrywidth=1,
#             subunitcolor='rgb(255, 255, 255)',
#             countrycolor='rgb(255, 255, 255)'
#         )
#     )
#     return fig

# def add_lng_traces(fig, lng_dict):
#     for month_idx in range(12):
#         for point_key, point_data in lng_dict.items():
#             entries = point_data['total_entries'][month_idx]
#             exits = point_data['total_exits'][month_idx]
#             lat, lon = point_data['coordinates']
#             size = max(entries, exits) ** 0.5  # Marker size based on the max value of entries or exits
#             color = entries - exits  # Color gradient based on the difference between entries and exits
#             marker = dict(size=size, color=color, colorscale='Viridis', colorbar=dict(title='Flow Difference'))
#             if entries == 0 and exits == 0:
#                 marker['color'] = 'black'
            
#             fig.add_trace(go.Scattergeo(
#                 lon=[lon],
#                 lat=[lat],
#                 text=f"{point_data['point_label']}<br>Entries: {entries}<br>Exits: {exits}",
#                 marker=marker,
#                 name=point_data['point_label'],
#                 visible=(month_idx == 0)  # Only show the first month by default
#             ))

# def add_slider_to_figure(fig):
#     steps = []
#     for month_idx in range(12):
#         step = dict(
#             method='update',
#             label=f'Month {month_idx + 1}',
#             args=[{
#                 'visible': [i // 3 == month_idx for i in range(36)]
#             }]
#         )
#         steps.append(step)
    
#     fig.update_layout(
#         sliders=[{
#             'active': 0,
#             'currentvalue': {
#                 'prefix': 'Month: '
#             },
#             'pad': {
#                 't': 50
#             },
#             'steps': steps
#         }]
#     )

# def add_country_click_interaction(fig, lng_dict):
#     """
#     Adds click interaction to the map for displaying LNG points of the clicked country.
    
#     Pre: fig is a Plotly figure object with LNG data traces. lng_dict is the dictionary containing LNG data.
#     Post: Returns the fig updated with the click interaction.
#     """
#     # Adding click interaction through JavaScript in Plotly's config
#     fig.update_layout(clickmode='event+select')
    
#     fig.add_trace(go.Scattergeo(
#         lon=[], lat=[], mode='markers+text', text=[], showlegend=False, textposition="top right", 
#         marker=dict(size=10, color='rgba(0,0,0,0)')
#     ))

#     fig['layout'].on_click = lambda trace, points, state: on_click(trace, points, state, lng_dict)
    
#     return fig

# def on_click(trace, points, state, lng_dict):
#     """
#     Handles the click event on the map and updates the display with LNG points of the clicked country.
    
#     Pre: trace, points, and state are provided by Plotly's event system. lng_dict contains the LNG data.
#     Post: Updates the display with the clicked country's LNG points and their data.
#     """
#     clicked_country = points[0].customdata[0]  # Assuming customdata contains the country information
#     lng_points = [point for point in lng_dict.values() if point['Country'] == clicked_country]
    
#     # Clear existing points
#     state['fig'].data[-1].lon = []
#     state['fig'].data[-1].lat = []
#     state['fig'].data[-1].text = []

#     for point in lng_points:
#         state['fig'].data[-1].lon.append(point['coordinates'][1])
#         state['fig'].data[-1].lat.append(point['coordinates'][0])
#         state['fig'].data[-1].text.append(point['point_label'])
    
#     # Add traces for the evolution of entries per point over the year
#     fig = make_subplots(rows=2, cols=1, subplot_titles=('LNG Points', 'Entries Evolution'))
    
#     fig.add_trace(state['fig'].data[-1], row=1, col=1)
    
#     for point in lng_points:
#         fig.add_trace(go.Scatter(
#             x=list(range(1, 13)), 
#             y=point['total_entries'], 
#             mode='lines+markers', 
#             name=point['point_label']
#         ), row=2, col=1)
    
#     fig.update_layout(height=800)
#     state['fig'] = fig

# # Example usage:
# # Assuming lng_points_df, lng_keys, and dates are already defined
# create_interactive_lng_map( lng_keys, lng_points_df, ['2019_01', '2019_02', '2019_03'], '../data/opData')


In [48]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Sample data
lng_dict = initialize_lng_dict(lng_points_df)
process_total_data(lng_dict, ['2019_01', '2019_02', '2019_03'], lng_keys, '../data/opData')


In [49]:
# Create the main map figure
def create_initial_map():
    fig = go.Figure()
    for point_key, point_data in lng_dict.items():
        lat, lon = point_data['coordinates']
        fig.add_trace(go.Scattergeo(
            lon=[lon],
            lat=[lat],
            text=point_data['point_label'],
            marker=dict(size=10),
            name=point_data['Country'],
            customdata=[point_data['Country']]
        ))
    fig.update_layout(
        geo=dict(
            projection_type='mercator',
            showland=True,
            landcolor='rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor='rgb(255, 255, 255)',
            countrycolor='rgb(255, 255, 255)'
        )
    )
    return fig

In [50]:
# Dash Layout
app.layout = html.Div([
    dcc.Graph(
        id='main-map',
        figure=create_initial_map(),
        style={'width': '70%', 'display': 'inline-block'}
    ),
    html.Div(id='sidebar',
             style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'})
])

# # Update sidebar
# @app.callback(
#     Output('sidebar', 'children'),
#     Input('main-map', 'clickData')
# )


SyntaxError: unexpected EOF while parsing (2430213836.py, line 16)

In [53]:
# Other solution for the sidebar
# clicked_country = "Spain"
# {k:point for k,point in lng_dict.items() if point['Country'] == clicked_country}

In [ ]:

def update_sidebar(clickData):
    if clickData is None:
        return html.Div('Click on a country to see related data')
    
    clicked_country = clickData['points'][0]['customdata'][0]
    lng_points = [point for point in lng_dict.items() if point['Country'] == clicked_country]
    
    # Create a table with the LNG points
    table = html.Table([
        html.Tr([html.Th('Point Label'), html.Th('Country')]),
        *[html.Tr([html.Td(point['point_label']), html.Td(point['Country'])]) for point in lng_points]
    ])

    fig = go.Figure()
    for k,data in lng_points.items():
        fig.add_trace(go.Scatter(
            x=list(range(1, 13)), 
            y=data['total_entries'], 
            mode='lines+markers', 
            name=data['point_label']
        ))

    fig.update_layout(
        title = f'LNG entries in {clicked_country} (2019)',
        xaxis_title = 'Month',
        yaxis_title = 'Entries (kWh/d)'
    )    

    return html.Div([
        html.H2(f'LNG Points in {clicked_country}'),
        table,
        dcc.Graph(figure=fig)
    ])

In [67]:
# TEST FROM mapCountry
# Plotly express dataframe
# clicked_country = "Spain"
# lng_points = [point for k,point in lng_dict.items() if point['Country'] == clicked_country]

In [66]:
# print(f"Country {clicked_country} has points : ")
# for point in lng_points:
#     print(f" - {point['point_label']} , is operational : {point['is_operational']}")

Country Spain has points : 
 - Barcelona , is operational : True
 - Bilbao , is operational : True
 - Huelva , is operational : True
 - Mugardos , is operational : True
 - Cartagena , is operational : True
 - Sagunto , is operational : True
 - TVB , is operational : False
 - Musel , is operational : True


In [69]:
# TEST FROM mapCountry

# data = []
# print(lng_points)
# for pt in lng_points:
#     months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
#     data.append({
#         'point_label': pt['point_label'],
#         'entries': pt['total_entries'],
#         'month' : months
#     })

# df = pd.DataFrame(data)
# df.head()

[{'point_label': 'Barcelona', 'coordinates': (41.35, 2.167), 'is_operational': True, 'Country': 'Spain', 'total_entries': [1952384514.0, 1839183428.0, 1869878711.0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'total_exits': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'point_label': 'Bilbao', 'coordinates': (43.333, -3.0), 'is_operational': True, 'Country': 'Spain', 'total_entries': [1398714753.0, 1271263781.0, 1629902486.0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'total_exits': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'point_label': 'Huelva', 'coordinates': (37.2, -6.95), 'is_operational': True, 'Country': 'Spain', 'total_entries': [1031197369.0, 1594347525.0, 1528345456.0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'total_exits': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'point_label': 'Mugardos', 'coordinates': (43.483, -8.25), 'is_operational': True, 'Country': 'Spain', 'total_entries': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'total_exits': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'point_label': 'Cartagena', 'coordinates': (37.6, -0.9

,point_label,entries,month
0,Barcelona,"[1952384514.0, 1839183428.0, 1869878711.0, 0, ...","[Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, ..."
1,Bilbao,"[1398714753.0, 1271263781.0, 1629902486.0, 0, ...","[Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, ..."
2,Huelva,"[1031197369.0, 1594347525.0, 1528345456.0, 0, ...","[Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, ..."
3,Mugardos,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, ..."
4,Cartagena,"[449526798.0, 461846458.0, 352514264.0, 0, 0, ...","[Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, ..."


In [ ]:
# RUN APP
if __name__ == '__main__':
    app.run_server(debug=True)

# Transmission points data

## Prepare Data
When data is prepared, add the preprocession steps to buildDataPerPointType

In [75]:
# CODE FROM buildDataPerPointType.py
# GOAL : Create aggregated data for intra-country flows

DIR = '../'
DATE = '2019_01'

######################
#    Data Loading    #
######################
# CODE FROM buildDataPerPointType.py
points_df = pd.read_csv(f'{DIR}/data/points_data.csv')
op_data = pd.read_csv(f'{DIR}/data/opData/op_data_{DATE}.csv')


In [78]:
######################
#    Data Processing #
######################
### Filtering Transmission points
trans_points_df = filter_point_type("Transmission",points_df,'point_type')
prod_points_df = filter_point_type("production",points_df,'point_type')

print(f"Total points : {len(points_df)}")
print(f"Transmission points : {len(trans_points_df)}")
print(f"Production points : {len(prod_points_df)}")

Total points : 772
Transmission points : 240
Production points : 43


In [ ]:
DATA_DIR = "../data/opData"

def load_data(date):
    """Load operational, points, operator points directions, and interconnections data."""
    operational_df = pd.read_csv(f"{DATA_DIR}/op_data_{date}.csv")
    points_df = pd.read_csv(f"{DATA_DIR}/points_data.csv")
    operator_points_directions_df = pd.read_csv(f"{DATA_DIR}/operator_points_dir.csv")
    interconnections_df = pd.read_csv(f'{DATA_DIR}/interconnections_data.csv')
    return operational_df, points_df, operator_points_directions_df, interconnections_df